In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col
import matplotlib as mp
import seaborn as snb
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
-- selecting the data of 2024
select rs.resultid, rc.name as racename,concat(d.forename,'',d.surname) as drivername, c.name as constructorname, s.status,rs.*
from 
playground.datathon.results rs
left join playground.datathon.races rc on rs.raceid = rc.raceid
left join playground.datathon.drivers d on rs.driverid = d.driverid
left join playground.datathon.constructors c on rs.constructorid = c.constructorid
left join playground.datathon.status s on rs.statusid = s.statusid
where rc.year = 2024
and rs.rank <> 0 

In [ ]:
-- 
select rc.name as racename,concat(d.forename,'',d.surname) as drivername, c.name as constructorname, s.status
,case when rs.grid = rs.position THEN 'No Change' WHEN rs.grid > rs.position THEN 'Forward' ELSE 'Backward' END AS POS_CHANGES
,rs.grid,rs.position as pos,rs.points pts,rs.laps,rs.milliseconds,rs.rank,rs.fastestlap
,rs.*
from 
playground.datathon.results rs
left join playground.datathon.races rc on rs.raceid = rc.raceid
left join playground.datathon.drivers d on rs.driverid = d.driverid
left join playground.datathon.constructors c on rs.constructorid = c.constructorid
left join playground.datathon.status s on rs.statusid = s.statusid
where UPPER(s.status) = 'FINISHED'
--and c.name = 'Mercedes'
--and rc.name = 'Australian Grand Prix'
order by rs.position


In [ ]:
-- Summary of constructor based on points, number of laps done by each driver
select c.name as constructorname,sum(rs.points),SUM(laps),count(d.driverid),
from 
playground.datathon.results rs
left join playground.datathon.races rc on rs.raceid = rc.raceid
left join playground.datathon.drivers d on rs.driverid = d.driverid
left join playground.datathon.constructors c on rs.constructorid = c.constructorid
left join playground.datathon.status s on rs.statusid = s.statusid
where UPPER(s.status) = 'FINISHED'
group by c.name
order by sum(rs.points) desc

In [ ]:
select rs.resultid, rc.name as racename,concat(d.forename,'',d.surname) as drivername, c.name as constructorname, s.status
,rs.grid,rs.position as pos ,rs.positiontext as post,rs.positionorder,rs.points,rs.laps,rs.milliseconds,rs.rank,rs.fastestlap
,rs.*
from 
playground.datathon.results rs
left join playground.datathon.races rc on rs.raceid = rc.raceid
left join playground.datathon.drivers d on rs.driverid = d.driverid
left join playground.datathon.constructors c on rs.constructorid = c.constructorid
left join playground.datathon.status s on rs.statusid = s.statusid
where UPPER(s.status) != 'FINISHED'
and rs.milliseconds is not null
--aND RS.POINTS != 0

In [ ]:
-- which circuits does Mercedes dominate?
SELECT  rc.circuitid, cr.name, d.driverid, sum(rs.points) as totalpoints
FROM 
PLAYGROUND.DATATHON.RESULTS rs 
left join PLAYGROUND.DATATHON.DRIVERS d on d.driverid = rs.driverid
left join PLAYGROUND.DATATHON.constructors c on c.constructorid = rs.constructorid
left join PLAYGROUND.datathon.races rc on rc.raceid = rs.raceid
left join PLAYGROUND.datathon.circuits cr on rc.circuitid = cr.circuitid
where c.name = 'Mercedes'
and rc.year = 2024
and rs.position is not null
--and rs.position <= 5
group by rc.circuitid, cr.name,d.driverid
order by sum(rs.points) desc



In [ ]:
-- Which circuits does Mercedes dominate in 2024?

SELECT  cr.name as Circuit, concat(d.forename,' ',d.surname) as Driver --,rc.raceid
, min(position) top_position
FROM 
PLAYGROUND.DATATHON.RESULTS rs 
left join PLAYGROUND.DATATHON.DRIVERS d on d.driverid = rs.driverid
left join PLAYGROUND.DATATHON.constructors c on c.constructorid = rs.constructorid
left join PLAYGROUND.datathon.races rc on rc.raceid = rs.raceid
left join PLAYGROUND.datathon.circuits cr on rc.circuitid = cr.circuitid
where c.name = 'Mercedes'
and rc.year = 2024
and rs.position is not null
and rs.position <= 3
group by cr.name,concat(d.forename,' ',d.surname)--,rc.raceid
--order by sum(rs.points) desc

In [ ]:
-- Who gains the most positions after qualifying?

select  concat(d.forename,' ',d.surname) as Driver, max(rs.grid - NVL(rs.position,99)) as POS_GAINED
from PLAYGROUND.datathon.results rs 
join  PLAYGROUND.datathon.qualifying q on rs.driverid = q.driverid 
left join PLAYGROUND.DATATHON.DRIVERS d on d.driverid = rs.driverid
left join PLAYGROUND.datathon.races rc on rc.raceid = rs.raceid
where 
rc.year = 2024
group by concat(d.forename,' ',d.surname)
--order by max(rs.grid - NVL(rs.position,99)) desc


In [ ]:
--pit_stops.csv — Strategy in Seconds

select avg(ps.milliseconds) as Average_PitStop_Time_MS,concat(d.forename,' ',d.surname) as Driver,c.name as Constructor
from
PLAYGROUND.DATATHON.PIT_STOPS ps 
left join PLAYGROUND.DATATHON.drivers d on ps.driverid = d.driverid
left join PLAYGROUND.DATATHON.results rs on rs.driverid = d.driverid
left join PLAYGROUND.DATATHON.constructors c on rs.constructorid = c.constructorid
group by concat(d.forename,' ',d.surname),c.name
order by avg(ps.milliseconds) desc

In [ ]:
--lap_times.csv — Micro-level Performance

select avg(lt.milliseconds) as AVG_LAP_TIME_MS, concat(d.forename,'',d.surname) as driver, c.name as constructor
from
PLAYGROUND.DATATHON.lap_times lt 
left join PLAYGROUND.DATATHON.drivers d on lt.driverid = d.driverid
left join PLAYGROUND.DATATHON.results rs on rs.driverid = d.driverid
left join PLAYGROUND.DATATHON.constructors c on rs.constructorid = c.constructorid
group by concat(d.forename,'',d.surname),c.name
order by avg(lt.milliseconds) 

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell13.to_pandas()

# Chart the data
st.subheader("Chance")
st.line_chart(my_df, x='YEAR', y='ROUND')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")

In [ ]:
# The Snowpark package is required for Python Worksheets. 
# You can add more packages by selecting them using the Packages control and then importing them.



def main(session: snowpark.Session):    
  #  session.use_database("EVENT")
    
    # Your code goes here, inside the "main" handler.
  #  tableName = 'information_schema.packages'
   # dataframe = session.table(tableName).filter(col("language") == 'python')

  # EVENT.DATATHON_2025_TEAM_EPSILON.RESULTS_NEW
    
    tableName = 'PLAYGROUND.DATATHON.RESULTS'
    dataframe = session.table(tableName).filter(col("rank") != 0)

    dataframe.show()

    # Return value will appear in the Results tab.
  #  return dataframe

    
    tableName2 = 'EVENT.DATATHON_2025_TEAM_EPSILON.RESULTS_NEW'
    dataframe2 = session.table(tableName2)

    dataframe2.show()

    # Return value will appear in the Results tab.
    return dataframe2

    
 

In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT 'FRIDAY' as SNOWDAY, 0.2 as CHANCE_OF_SNOW
UNION ALL
SELECT 'SATURDAY',0.5
UNION ALL 
SELECT 'SUNDAY', 0.9;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")